In [1]:
#Install yfinance
%pip install yfinance

In [2]:
#Import Libraries
import pandas as pd
import yfinance as yf
from datetime import datetime

#Set a Date Range for 6 months before
start_date = datetime.now() - pd.DateOffset(months=6)
end_date = datetime.now()

#List of Tickers for Apple, Microsoft, Netflix, and Google
tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG']

#Define an empty list 
ds_list = []

#Download the data from Yahoo Finance
for ticker in tickers:
    data_set = yf.download(ticker, start=start_date, end=end_date)
    ds_list.append(data_set)

ds = pd.concat(ds_list, keys=tickers, names=['Ticker', 'Date'])
print(ds.head())

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-03-15  151.190002  153.250000  149.919998  152.990005  152.572647   
       2023-03-16  152.160004  156.460007  151.639999  155.850006  155.424850   
       2023-03-17  156.080002  156.740005  154.279999  155.000000  154.577164   
       2023-03-20  155.070007  157.820007  154.149994  157.399994  156.970612   
       2023-03-21  157.320007  159.399994  156.539993  159.279999  158.845490   

                     Volume  
Ticker Date                  
AAPL   2023-03-15  77167900  
       2023-03-16  76161100  
       2023-03-17  98944600  
      

In [3]:
#reset the index in the dataframe
ds.reset_index(inplace=True)
print(ds.head())

  Ticker       Date        Open        High         Low       Close  \
0   AAPL 2023-03-15  151.190002  153.250000  149.919998  152.990005   
1   AAPL 2023-03-16  152.160004  156.460007  151.639999  155.850006   
2   AAPL 2023-03-17  156.080002  156.740005  154.279999  155.000000   
3   AAPL 2023-03-20  155.070007  157.820007  154.149994  157.399994   
4   AAPL 2023-03-21  157.320007  159.399994  156.539993  159.279999   

    Adj Close    Volume  
0  152.572647  77167900  
1  155.424850  76161100  
2  154.577164  98944600  
3  156.970612  73641400  
4  158.845490  73938300  


In [4]:
%pip install -U kaleido

Note: you may need to restart the kernel to use updated packages.


In [5]:
#Performance analysis of in the stock market for past 6 months

#import plotly
import plotly.express as px

#Create a line chart in dark mode
px.defaults.template = "plotly_dark"

#Create a line chart
figStckMktAn = px.line(ds, x='Date', 
               y='Close', 
              color='Ticker', 
              title="Stock Market Performance for the Last 6 Months")
figStckMktAn.show()

#Save the chart as png file using keleido
figStckMktAn.write_image("Stock Market Performance.png")

In [6]:
#Faceted area chart for six months
figFctArCht = px.area(ds, x='Date', y='Close', color='Ticker',
              facet_col='Ticker',
              labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
              title='Stock Prices for Apple, Microsoft, Netflix, and Google')
figFctArCht.show()

#Save the chart as png file using keleido
figFctArCht.write_image("Faceted area chart.png")

In [7]:
#Analyze moving averages
ds['MA10'] = ds.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
ds['MA20'] = ds.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in ds.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

Moving Averages for AAPL
           MA10        MA20
0           NaN         NaN
1           NaN         NaN
2           NaN         NaN
3           NaN         NaN
4           NaN         NaN
..          ...         ...
122  183.624998  180.328999
123  183.699998  180.407500
124  183.310999  180.249500
125  182.320000  180.087500
126  181.129001  180.046000

[127 rows x 2 columns]
Moving Averages for GOOG
           MA10        MA20
381         NaN         NaN
382         NaN         NaN
383         NaN         NaN
384         NaN         NaN
385         NaN         NaN
..          ...         ...
503  135.453000  132.836501
504  136.158000  133.215001
505  136.586002  133.427001
506  136.787001  133.788501
507  136.993002  134.282501

[127 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
127         NaN         NaN
128         NaN         NaN
129         NaN         NaN
130         NaN         NaN
131         NaN         NaN
..          ...         ...
249  329.

In [8]:
#Visualize moving averages
for ticker, group in ds.groupby('Ticker'):
    figMovAvg = px.line(group, x='Date', y=['Close', 'MA10', 'MA20'], 
                  color_discrete_sequence=['red', 'blue', 'green'],
                  title=f"{ticker} Moving Averages")
    figMovAvg.show()

#Save multiple charts as png files using keleido
    figMovAvg.write_image(f"{ticker} Moving Averages.png")


In [9]:
#Analyze volatility of the companies
ds['Volatility'] = ds.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
figVol = px.line(ds, x='Date', y='Volatility', 
              color='Ticker', 
              title='Volatility of All Companies')
figVol.show()

#Save the chart as png file using keleido
figVol.write_image("Volatility.png")


In [10]:
#Analyze the correlation between the companies (Netflix and Microsoft)

# create a DataFrame with the stock prices of Netflix and Microsoft
netflix = ds.loc[ds['Ticker'] == 'NFLX', ['Date', 'Close']].rename(columns={'Close': 'NFLX'})
microsft = ds.loc[ds['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})
ds_corr = pd.merge(netflix, microsft, on='Date')

# create a scatter plot to visualize the correlation
figCorNFLXnMSFT = px.scatter(ds_corr, x='NFLX', y='MSFT', 
                 trendline='ols', 
                 title='Correlation between Netflix and Microsoft')
figCorNFLXnMSFT.show()

#Save the chart as png file using keleido
figCorNFLXnMSFT.write_image("Correlation between Netflix and Microsoft.png")

In [55]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import plotly.graph_objects as go
import numpy as np

In [56]:
# Define the target column you want to predict
target_column = 'Close'

# Define the feature columns you want to use for prediction
feature_columns = ['Open', 'High', 'Low', 'Adj Close', 'Volume']

In [57]:
# Define the number of future days to predict
num_future_days = 5

In [58]:
# List to store the future predictions for all tickers
all_ticker_predictions = []

In [59]:
# Iterate over the unique tickers in the dataset
unique_tickers = ds['Ticker'].unique()

In [60]:
for ticker in unique_tickers:
    # Filter the data for the current ticker
    ticker_data = ds[ds['Ticker'] == ticker]

    # Split the data into training and testing sets
    X = ticker_data[feature_columns].values
    y = ticker_data[target_column].values

    # Initialize and train a Linear Regression model
    model = LinearRegression()
    model.fit(X, y)

    # Predict future stock prices for the next 'num_future_days' days
    last_data_point = X[-1].reshape(1, -1)  # Reshape for prediction
    future_predictions = []

    for _ in range(num_future_days):
        future_prediction = model.predict(last_data_point)
        future_predictions.append(future_prediction)
        # Update the last_data_point with the predicted value for the next iteration
        last_data_point[0][-1] = future_prediction

    # Create a DataFrame for future predictions
    future_dates = pd.date_range(start=ticker_data['Date'].iloc[-1], periods=num_future_days, freq='D')
    future_results_df = pd.DataFrame({'Date': future_dates, 'Predicted': np.array(future_predictions).flatten()})
    future_results_df['Ticker'] = ticker  # Add the ticker column

    # Append the future predictions for the current ticker to the list
    all_ticker_predictions.append(future_results_df)

In [61]:
# Concatenate the future predictions for all tickers into a single DataFrame
all_predictions_ds = pd.concat(all_ticker_predictions)

In [62]:
# Concatenate the future predictions for all tickers into a single DataFrame
all_predictions_ds = pd.concat(all_ticker_predictions)


# Create time series graphs for future predictions for all tickers
figs = []

In [63]:
for ticker in unique_tickers:
    ticker_predictions = all_predictions_ds[all_predictions_ds['Ticker'] == ticker]
    
    # Create a time series graph for future predictions
    fig = go.Figure()

    # Add future predicted values as a line
    fig.add_trace(go.Scatter(x=ticker_predictions['Date'], y=ticker_predictions['Predicted'], mode='lines', name='Predicted'))

    # Customize the layout for future predictions
    fig.update_layout(
        title=f'Predicted Stock Prices for {ticker} (Next {num_future_days} Days)',
        xaxis_title='Date',
        yaxis_title='Price',
    )

    figs.append(fig)

In [64]:
# Display the individual time series graphs for future predictions
for fig in figs:
    fig.show()